In [49]:
import mido
import numpy as np

MAX_NUM_OF_CHANNELS = 16
DRUM_CHANNEL = 9

# return true if the message is a note
def is_note(msg):
    return msg.type == 'note_on' or msg.type == 'note_off'

# return true if the message indicates the start of a note
# return false if the message indicates the end of a note
def is_note_on(msg):
    return msg.type == 'note_on' and msg.velocity != 0

def is_drum(msg):
    return msg.channel == DRUM_CHANNEL

# get the time and tick indicating the bars, and their beat and tempo
def get_bars_info(midi_file):
    
    ticks_per_beat = midi_file.ticks_per_beat

    total_tick = 0
    total_time = 0
    rhythm_events = []

    # store the time signature and tempo change events as rhythm_events
    for msg in midi_file:

        # sum up the tick/time
        if msg.time != 0:
            total_tick += mido.second2tick(msg.time, ticks_per_beat, current_tempo)
            total_time += msg.time

        # store meta message of time signature and tempo with current tick
        if msg.is_meta:

            if msg.type == 'time_signature':
                time_signature_quarter = msg.numerator / (msg.denominator / 4)
                rhythm_events.append([total_tick, -1, time_signature_quarter])

            if msg.type == 'set_tempo':
                current_tempo = msg.tempo
                rhythm_events.append([total_tick, current_tempo, -1])
    
    # store the tick of the end of midi
    rhythm_events.append([total_tick, -1, -1])

    cumulative_time = 0
    cumulative_tick = 0
    prev_tick = 0
    current_tempo = -1
    current_ts_quarter = -1
    bars_time = []
    bars_tick = []
    bars_beat = []
    bars_tempo = []

    # append the time/tick at each bar according to 'rhythm_events'
    for rhythm_event in rhythm_events:

        # detect rhythm change, then conclude the bars before the change
        if rhythm_event[0] != prev_tick:

            tick_diff = rhythm_event[0] - prev_tick
            prev_tick = rhythm_event[0]

            # conclude the bar time/tick by dividing the bar length
            # given total tick, and constant beat and tempo, the bars can be separated by dividing evenly
            num_of_bar = tick_diff / ticks_per_beat
            num_of_bar_2 = num_of_bar
            while num_of_bar > 0:
                bar_tick = (num_of_bar_2 - num_of_bar) * ticks_per_beat
                bars_time.append(mido.tick2second(bar_tick, ticks_per_beat, current_tempo) + cumulative_time)
                bars_tick.append(bar_tick + cumulative_tick)
                bars_beat.append(current_ts_quarter)
                bars_tempo.append(current_tempo)
                num_of_bar -= current_ts_quarter            

            cumulative_time += mido.tick2second(tick_diff, ticks_per_beat, current_tempo)
            cumulative_tick += tick_diff

        # update the time signature and time tempo events
        if rhythm_event[1] != -1:
            current_tempo = rhythm_event[1]
        if rhythm_event[2] != -1:
            current_ts_quarter = rhythm_event[2]

    return bars_time, bars_tick, bars_beat, bars_tempo

# get the details of notes in each bar
# i.e. bars_note_info[bar index] = [start tick of the bar, end tick of the bar, bar_note_info]
# i.e. bar_note_info[note index] = [channel index, note no., triggered time (in tick), 1 = 'note_on' / 0 = 'note_off']
def get_bars_note_info(midi_file):

    _, bars_tick, bars_beat, _ = get_bars_info(midi_file)

    ticks_per_beat = midi_file.ticks_per_beat
    current_tempo = -1
    current_ts_quarter = -1

    total_tick = 0
    bar_start_tick = bars_tick.pop(0)
    bar_note_info = []
    bars_note_info = []

    # store the time signature and tempo change events as rhythm_events
    for msg in midi_file:

        # sum up the tick/time
        if msg.time != 0:        
            total_tick += mido.second2tick(msg.time, ticks_per_beat, current_tempo)
            if len(bars_tick) != 0 and total_tick >= bars_tick[0]:

                # find the suitable bar for placing current note
                bars_shift_idx = 0
                while len(bars_tick) > (bars_shift_idx + 1) and total_tick >= bars_tick[bars_shift_idx + 1]:
                    bars_shift_idx += 1

                bars_note_info.append([bar_start_tick, bars_tick[0] - bar_start_tick, bar_note_info, bars_beat[0]])
                bar_note_info = []
                bar_start_tick = bars_tick.pop(0)
                bars_beat.pop(0)

                # put back the shifted (empty) bars
                for pad_bar_idx in range(bars_shift_idx):
                    bars_note_info.append([bar_start_tick, bars_tick[0] - bar_start_tick, bar_note_info, bars_beat[0]])
                    bar_note_info = []
                    bar_start_tick = bars_tick.pop(0)
                    bars_beat.pop(0)


        # store meta message of time signature and tempo with current tick
        if msg.is_meta:

            if msg.type == 'time_signature':
                current_ts_quarter = msg.numerator / (msg.denominator / 4)

            if msg.type == 'set_tempo':
                current_tempo = msg.tempo

        if not msg.is_meta:

            if is_note(msg):

                if is_note_on(msg):
                    bar_note_info.append([msg.channel, msg.note, total_tick, 1, msg.velocity])

                else:
                    bar_note_info.append([msg.channel, msg.note, total_tick, 0, msg.velocity])
    return bars_note_info

# get rhythm pattern in each bar of each channel
# i.e. tracks_rhythm_pattern[channel index, bar index] = [[note1 start_time, note1 duration], [note2 start_time, note2 duration], ...]
def get_rhythm_pattern(bars_note_info):

    MAX_NUM_OF_CHANNELS = 16
    MAX_NUM_OF_NOTE_IN_BAR = 64
    tracks_rhythm_pattern = np.zeros([MAX_NUM_OF_CHANNELS, len(bars_note_info), MAX_NUM_OF_NOTE_IN_BAR, 5]) - 1
    # print(tracks_rhythm_pattern.shape)


    prev_bar_notes = []

    for idx, bar_note_info in enumerate(bars_note_info):

        if len(prev_bar_notes) != 0:
            for prev_bar_note in prev_bar_notes:
                append_idx = 0
                while append_idx < 64:
                    if tracks_rhythm_pattern[prev_bar_note[0], idx, append_idx, 0] == -1:
                        tracks_rhythm_pattern[prev_bar_note[0], idx, append_idx] = [0, prev_bar_note[1], 0, 0, 0]
                        break
                    append_idx += 1
            prev_bar_notes = []

        # print(idx, bar_note_info[2])
        for note in bar_note_info[2]:
            if note[3]:
                append_idx = 0
                while append_idx < 64:
                    if tracks_rhythm_pattern[note[0], idx, append_idx, 0] == -1:
                        tracks_rhythm_pattern[note[0], idx, append_idx, 0] = note[2] - bar_note_info[0]
                        tracks_rhythm_pattern[note[0], idx, append_idx, 1] = -2 - note[1]
                        break
                    append_idx += 1
            else:
                # print(tracks_rhythm_pattern[note[0], idx, :, 1], -2 - note[1])
                relative_idx = np.nonzero(tracks_rhythm_pattern[note[0], idx, :, 1] == -2 - note[1])[0][0]
                tracks_rhythm_pattern[note[0], idx, relative_idx, 1] = note[2] - bar_note_info[0] - tracks_rhythm_pattern[note[0], idx, relative_idx, 0]
                # print(tracks_rhythm_pattern[note[0], idx, relative_idx])

        for channel_idx in range(MAX_NUM_OF_CHANNELS):

            temp_idx = 0

            while temp_idx < MAX_NUM_OF_NOTE_IN_BAR and tracks_rhythm_pattern[channel_idx, idx, temp_idx, 1] != -1:
                if tracks_rhythm_pattern[channel_idx, idx, temp_idx, 1] < -1:
                    prev_bar_notes.append([channel_idx, tracks_rhythm_pattern[channel_idx, idx, temp_idx, 1]])
                    tracks_rhythm_pattern[channel_idx, idx, temp_idx, 1] = bar_note_info[1] - tracks_rhythm_pattern[note[0], idx, relative_idx, 0]
                    # print(tracks_rhythm_pattern[channel_idx, idx, temp_idx])
                tracks_rhythm_pattern[channel_idx, idx, temp_idx] /= bar_note_info[1]
                temp_idx += 1
        # print(prev_bar_notes)
    
    return tracks_rhythm_pattern

def get_midi_info(bars_note_info):
    midi_info = np.empty((len(bars_note_info), MAX_NUM_OF_CHANNELS), dtype=np.ndarray)
    for i in range(midi_info.shape[0]):
        for j in range(midi_info.shape[1]):
            midi_info[i, j] = np.empty((0, 7))


    prev_bar_notes = []

    for idx, bar_note_info in enumerate(bars_note_info):

        if len(prev_bar_notes) != 0:
            for prev_bar_note in prev_bar_notes:
                midi_info[idx, prev_bar_note[0]] = np.vstack((midi_info[idx, prev_bar_note[0]], [prev_bar_note[2], prev_bar_note[3], 0, prev_bar_note[1], 0, 0, 1]))
            prev_bar_notes = []

        for note in bar_note_info[2]:
            if note[3]:
                midi_info[idx, note[0]] = np.vstack((midi_info[idx, note[0]], [note[1], note[4], note[2] - bar_note_info[0], -2 - note[1], 0, 0, 0]))
            else:
                relative_idx = np.nonzero(midi_info[idx, note[0]][:, 3] == -2 - note[1])[0][0]
                midi_info[idx, note[0]][relative_idx, 3] = note[2] - bar_note_info[0] - midi_info[idx, note[0]][relative_idx, 2]

        for channel_idx in range(MAX_NUM_OF_CHANNELS):

            for i, info in enumerate(midi_info[idx, channel_idx]):
                if info[3] < -1:
                    prev_bar_notes.append([channel_idx, info[3], info[0], info[1]])
                    midi_info[idx, channel_idx][i, 3] = bar_note_info[1] - info[2]
                    midi_info[idx, channel_idx][i, 6] = 1
            midi_info[idx, channel_idx][:, 2:4] /= bar_note_info[1]
            midi_info[idx, channel_idx][:, 4:6] = midi_info[idx, channel_idx][:, 2:4] * bar_note_info[3]
    return midi_info

# turn numeric notes into english notes, octave info not preserved
def note_int2char(note):
    return {
        '0': 'C', '1': 'Db', '2': 'D', '3': 'Eb',
        '4': 'E', '5': 'F', '6': 'F#', '7': 'G',
        '8': 'Ab', '9': 'A', '10': 'Bb', '11': 'B'
    }.get(str(note % 12), 'undef')

# define the chords and its relative position
chords = np.empty((0, 12), dtype=np.int8)

chords_entries = np.array([
    # C1
    [6, -1, 0, -4, -4, 0, -1, -5, -1, -1, -3, -3],
    # C5
    [5, -1, 0, -4, -4, 0, -1, 5, -1, -1, -3, -3],
    # C
    [5, -3, 0, -4, 4, 0, -3, 4, -3, 1, -4, -4],
    # Cm
    [5, -3, 0, 4, -4, 0, -3, 4, 1, 0, -4, -4],
    # C7
    [5, -3, 0, -3, 3, 0, -3, 3, 1, 1, 3, -4],
    # CMaj7
    [5, -3, 0, -3, 3, 0, -3, 3, -3, 1, -3, 3],
    # Cmin7
    [5, -3, 0, 3, -3, 0, -3, 3, 1, 0, 3, -1],
    # Cdim
    [5, 0, -3, 4, -3, -3, 4, -4, 0, -3, -3, -3],
], dtype=np.int8)

for chords_entry in chords_entries:
    for i in range(12):
        chords = np.vstack((chords, np.roll(chords_entry, i)))

# name the chord
def name_chord(chord):
    return {
        '0': note_int2char(chord) + '1',
        '1': note_int2char(chord) + '5',
        '2': note_int2char(chord),
        '3': note_int2char(chord) + 'm',
        '4': note_int2char(chord) + '7',
        '5': note_int2char(chord) + 'M7',
        '6': note_int2char(chord) + 'm7',
        '7': note_int2char(chord) + 'dim'
    }.get(str(chord // 12), 'undef')

# chords to key, '[C, Db, D, ..., B, Cm, Dbm, ..., Bbm, Bm]'
chord_to_key = np.empty((0, 24), dtype=np.int8)

chord_to_key_entries = np.array([
    # C note only
    [2, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 2, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0],
    # C power chords
    [2, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0],
    # C Major chords
    [2, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0],
    # C Minor chords
    [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
    # C Dominant 7th chords
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    # C Major 7th chords
    [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
    # C Minor 7th chords
    [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
    # C Diminished chords
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
], dtype=np.int8)

for chord_to_key_entry in chord_to_key_entries:
    for i in range(12):
        chord_to_key = np.vstack((chord_to_key, np.concatenate((np.roll(chord_to_key_entry[0:12], i), np.roll(chord_to_key_entry[12:24], i)))))

# name the key
def name_key(key):
    return {
        '0': note_int2char(key) + ' Major',
        '1': note_int2char(key) + ' minor',
    }.get(str(key // 12), 'undef')

# get chords of a key
def key_to_chords(key):
    return np.nonzero(np.transpose(chord_to_key)[np.argmax(key)])

def get_chords(midi_info):
    chords_predict = np.zeros(midi_info.shape[0], dtype=np.uint8)
    for bar_idx in range(midi_info.shape[0]):
        notes_stat = np.zeros(12, dtype=np.float)
        for channel_idx in range(MAX_NUM_OF_CHANNELS):
            if channel_idx != DRUM_CHANNEL:
                for info in midi_info[bar_idx, channel_idx]:
                    notes_stat[int(info[0] % 12)] += info[3]
        chords_score = np.dot(chords, notes_stat)
        chords_predict[bar_idx] = np.argmax(chords_score)
    #     with np.printoptions(precision=3, suppress=True):
    #         print(["%s: %f"%(note_int2char(idx), note) for idx, note in enumerate(notes_stat) if note > 0])
    #         print(name_chord(chord_predict))
    #         print([name_chord(chord) for chord in np.argsort(chords_score)[-10:]])
    return chords_predict

def get_key(chords_predict):
    chords_stat = np.zeros(chords.shape[0], dtype=np.int)
    for chord_predict in chords_predict:
        chords_stat[chord_predict] += 1
    key_stat = np.dot(chords_stat, chord_to_key)
    key_predict = np.argmax(key_stat)
    key_acc = np.sum(chords_stat[key_to_chords(key_stat)]) / np.sum(chords_stat)
    return key_predict, key_acc, chords_stat

def get_msg_non_note(midi_file):
    msg_non_note = np.empty((0,2))
    total_tick = 0
    ticks_per_beat = midi_file.ticks_per_beat
    current_tempo = 0

    for msg in midi_file:
        if msg.time != 0:
            total_tick += mido.second2tick(msg.time, ticks_per_beat, current_tempo)

        if msg.is_meta:
            # update tempo for 'second2tick'
            if msg.type == 'set_tempo':
                current_tempo = msg.tempo
            msg_non_note = np.vstack((msg_non_note, np.array([total_tick, msg])))
        else:
            if not is_note(msg):
                msg_non_note = np.vstack((msg_non_note, np.array([total_tick, msg])))
    return msg_non_note

def info2midi(midi_info, msg_non_note, ticks_per_beat):

    UINT32_MAX = 4294967296 - 1

    channels_block = np.empty((MAX_NUM_OF_CHANNELS), dtype=np.object)
    for i in range(MAX_NUM_OF_CHANNELS):
        channels_block[i] = np.empty((0, 5), dtype=np.uint32)

    ticks_per_beat = midi_file.ticks_per_beat

    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        for bar_idx in range(midi_info.shape[0]):
            bar_tick = ticks_per_beat * bars_beat[bar_idx]
            for info in midi_info[bar_idx, channel_idx]:
                if info[6] == 0:
                    channels_block[channel_idx] = np.vstack((channels_block[channel_idx], [(bar_idx + info[2]) * bar_tick, 0, channel_idx, info[0], info[1]]))
                    channels_block[channel_idx] = np.vstack((channels_block[channel_idx], [(bar_idx + info[2] + info[3]) * bar_tick, 1, channel_idx, info[0], info[1]]))
                else:
                    if info[2] == 0:
                        channels_block[channel_idx] = np.vstack((channels_block[channel_idx], [(bar_idx + info[2] + info[3]) * bar_tick, 1, channel_idx, info[0], info[1]]))
                    else:
                        channels_block[channel_idx] = np.vstack((channels_block[channel_idx], [(bar_idx + info[2]) * bar_tick, 0, channel_idx, info[0], info[1]]))
            channels_block[channel_idx] = channels_block[channel_idx][channels_block[channel_idx][:,0].argsort()]

    first_msg = np.zeros((17, 2), dtype=np.uint32)
    # initialize the comparison array
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        if channels_block[channel_idx].shape[0] != 0:
            first_msg[channel_idx, 1] = channels_block[channel_idx][0, 0]
        else:
            first_msg[channel_idx, 1] = UINT32_MAX
    if msg_non_note.shape[0] != 0:
        first_msg[16, 1] = msg_non_note[0, 0]
    else:
        first_msg[16, 1] = UINT32_MAX


    midi_file_new = mido.MidiFile(type=0)
    midi_file_new.ticks_per_beat = midi_file.ticks_per_beat
    track_new = mido.MidiTrack()
    midi_file_new.tracks.append(track_new)

    total_ticks = 0

    while True:
        channel_to_append = np.argmin(first_msg[:, 1])
        if first_msg[channel_to_append, 1] == UINT32_MAX:
            break
        if channel_to_append < 16:
            info = channels_block[channel_to_append][first_msg[channel_to_append, 0]]

            tick = (int)(info[0])
            msg = mido.Message('note_on', channel=(int)(info[2]), note=(int)(info[3]), velocity=(int)(info[4]), time=0)
            if info[1] == 1:
                msg = mido.Message('note_off', channel=(int)(info[2]), note=(int)(info[3]), velocity=(int)(info[4]), time=0)

            if tick != total_ticks:
                msg.time = tick - total_ticks
                total_ticks = tick

            track_new.append(msg)

            first_msg[channel_to_append, 0] += 1
            if first_msg[channel_to_append, 0] < channels_block[channel_to_append].shape[0]:
                first_msg[channel_to_append, 1] = channels_block[channel_to_append][first_msg[channel_to_append, 0], 0]
            else:
                first_msg[channel_to_append, 1] = UINT32_MAX
        else:
            info = msg_non_note[first_msg[channel_to_append, 0]]

            tick = (int)(info[0])        
            msg = info[1]
            msg.time = 0

            if tick != total_ticks:
                msg.time = tick - total_ticks
                total_ticks = tick

            track_new.append(msg)

            first_msg[channel_to_append, 0] += 1
            if first_msg[channel_to_append, 0] < msg_non_note.shape[0]:
                first_msg[channel_to_append, 1] = msg_non_note[first_msg[channel_to_append, 0], 0]
            else:
                first_msg[channel_to_append, 1] = UINT32_MAX
    return midi_file_new
            

In [50]:
# input the file name
midi_string = 'sample1'
midi_file = mido.MidiFile('../sample_audio/%s.mid' % midi_string)

# get the beat and tempo in each bar
_, bars_tick, bars_beat, bars_tempo = get_bars_info(midi_file)

# separate notes to their relative bars
bars_note_info = get_bars_note_info(midi_file)

# generate the rhythm pattern[channel index, bar index]
# rhythm_pattern = get_rhythm_pattern(bars_note_info)

midi_info = get_midi_info(bars_note_info)

chords_predict = get_chords(midi_info)

key_predict, key_acc, chords_stat = get_key(chords_predict)

msg_non_note = get_msg_non_note(midi_file)

with np.printoptions(precision=3, suppress=True):
#     print(bars_tick)
#     print([bar_note_info[3] for bar_note_info in bars_note_info])
    print(midi_info[:,1][0:2])
    print(chords_predict[0:30])
    print([name_chord(chord_predict) for chord_predict in chords_predict[0:30]])
    print('\nKey: %s (accuracy: %f)' % (name_key(key_predict), key_acc))
    print('\nChord statistics')
    for chord_stat_idx, chord_stat in enumerate(chords_stat):
        if chord_stat != 0:
            print('%s\t: %d' % (name_chord(chord_stat_idx), chord_stat))
    print(msg_non_note)
    
    print(bars_beat)
    print(bars_tick)

[array([[38. , 88. ,  0.5,  0.5,  2. ,  2. ,  1. ]])
 array([[38.   , 88.   ,  0.   ,  0.002,  0.   ,  0.006,  1.   ],
       [38.   , 88.   ,  0.   ,  0.184,  0.   ,  0.738,  0.   ],
       [50.   , 88.   ,  0.188,  0.059,  0.75 ,  0.237,  0.   ],
       [38.   , 88.   ,  0.25 ,  0.184,  1.   ,  0.738,  0.   ],
       [50.   , 88.   ,  0.438,  0.059,  1.75 ,  0.237,  0.   ],
       [38.   , 88.   ,  0.5  ,  0.184,  2.   ,  0.738,  0.   ],
       [50.   , 88.   ,  0.688,  0.059,  2.75 ,  0.237,  0.   ],
       [38.   , 88.   ,  0.75 ,  0.184,  3.   ,  0.738,  0.   ],
       [50.   , 88.   ,  0.938,  0.059,  3.75 ,  0.237,  0.   ]])]
[ 2  2  2  2  2 14 38 43 29 21 33 43 57 38 38 43 81 33 33  7 53  2  2  2
  2  2  2  7 29  9]
['D1', 'D1', 'D1', 'D1', 'D1', 'D5', 'Dm', 'Gm', 'F', 'A5', 'A', 'Gm', 'A7', 'Dm', 'Dm', 'Gm', 'Am7', 'A', 'A', 'G1', 'F7', 'D1', 'D1', 'D1', 'D1', 'D1', 'D1', 'G1', 'F', 'A1']

Key: D minor (accuracy: 0.888199)

Chord statistics
D1	: 34
G1	: 5
A1	: 4
D5	: 1
G5	: 5


In [51]:
midi_file_new = info2midi(midi_info, msg_non_note, midi_file.ticks_per_beat)

import time    

# use 'print(mido.get_output_names())' to get your port's name and type it in the 'mido.open_output' bracket
port = None
port = mido.open_output('Microsoft GS Wavetable Synth 0')
play_audio = True

try:

    for msg in midi_file_new:
        
        # delay for playing
        if play_audio:
            time.sleep(msg.time)
            if not msg.is_meta:
                port.send(msg)
#                 print(msg)

    if port is not None:
        port.close()
    print('End of midi')
    
# action after pressing stop
except KeyboardInterrupt:
    
    if port is not None:
        port.close()
    print('Forced stop')

Forced stop


In [65]:
# keys notes, '[C, Db, D, ..., B, Cm, Dbm, ..., Bbm, Bm]'
keys_notes = np.empty((0, 12), dtype=np.int8)

keys_notes_entries = np.array([
    # C Major
    [2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1],
    # C minor
    [2, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1],
], dtype=np.int8)

for keys_notes_entry in keys_notes_entries:
    for i in range(12):
        keys_notes = np.vstack((keys_notes, np.roll(keys_notes_entry[0:12], i)))

# chords notes, '[C, Db, D, ..., B, Cm, Dbm, ..., Bbm, Bm]'
chords_notes = np.empty((0, 12), dtype=np.int8)

chords_notes_entries = np.array([
    # C1
    [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    # C5
    [2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    # C
    [2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1],
    # Cm
    [2, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1],
    # C7
    [2, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0],
    # CMaj7
    [2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1],
    # Cmin7
    [2, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0],
    # Cdim
    [2, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
], dtype=np.int8)

for chords_notes_entry in chords_notes_entries:
    for i in range(12):
        chords_notes = np.vstack((chords_notes, np.roll(chords_notes_entry, i)))
        
# get each note scores in a bar based on only its note (higher score = more related to the key)
def get_bar_notes_priority(bar_info, key_predict, chord_predict):
    notes_score = keys_notes[key_predict] + chords_notes[chord_predict]
    bar_notes_priority = np.zeros(bar_info.shape[0], dtype=np.uint8)
    for idx, note_info in enumerate(bar_info):
        bar_notes_priority[idx] = notes_score[(int)(note_info[0] % 12)]
    return bar_notes_priority

# example
example_key_predict = 14
example_chord_predict = 57
example_bar_info = np.array([
    [41, 88, 0, 0.184, 0, 0.738, 0],
    [53, 88, 0.188, 0.059, 0.75, 0.237, 0],
    [41, 88, 0.25, 0.184, 1, 0.738, 0],
    [53, 88, 0.438, 0.059, 1.75, 0.237, 0],
    [38, 88, 0.5, 0.184, 2, 0.738, 0],
    [50, 88, 0.688, 0.059, 2.75, 0.237, 0],
    [38, 88, 0.75, 0.184, 3, 0.738, 0],
    [50, 88, 0.938, 0.059, 3.75, 0.237, 0],
])
with np.printoptions(precision=3, suppress=True):
    print("key:\t%s" % name_key(example_key_predict))
    print("bar:\t%s" % name_chord(example_chord_predict))
    print("note:\t%s" % example_bar_info[:, 0])
    print("prior:\t%s" % get_bar_notes_priority(example_bar_info, example_key_predict, example_chord_predict))

key:	D minor
bar:	A7
note:	[41. 53. 41. 53. 38. 50. 38. 50.]
prior:	[1 1 1 1 3 3 3 3]


In [66]:
# add pitch to notes with -1 pitch
# analysis by "median note over bar", "key" and "bar chord"
def bar_add_pitch(bar_info, key_predict, chord_predict):
    avg_pitch = np.median([note for note in bar_info[:, 0] if note != -1])
    notes_score = keys_notes[key_predict] + chords_notes[chord_predict]

    SEARCH_PITCH_RANGE = 12
    min_pitch = (int)(np.max((0, np.min((np.floor(avg_pitch - SEARCH_PITCH_RANGE), np.min([note for note in bar_info[:, 0] if note != -1]))))))
    max_pitch = (int)(np.min((127, np.max((np.floor(avg_pitch + SEARCH_PITCH_RANGE), np.max(bar_info[:, 0]))))))

    notes_priority = np.zeros(max_pitch - min_pitch + 1)
    for i in range(0, max_pitch - min_pitch + 1):
        notes_priority[i] = 1 / (np.abs(avg_pitch - (i + min_pitch)) + SEARCH_PITCH_RANGE) * notes_score[(i + min_pitch) % 12]

    avail_notes = np.argsort(-notes_priority) + min_pitch

    for empty_idx in np.argwhere(bar_info[:, 0] == -1).flatten():
        notes_on = []
        for note_info in [note_info for note_info in bar_info if note_info[0] != -1]:
            bool_overlap = not(note_info[2] >= (bar_info[empty_idx, 2] + bar_info[empty_idx, 3]) or (note_info[2] + note_info[3]) <= bar_info[empty_idx, 2])
            if (note_info[0] not in notes_on and bool_overlap):
                notes_on.append(note_info[0])
        i = 0
        while (avail_notes[i] in notes_on):
            i += 1
            if i == avail_notes.shape[0]:
                i = 0
                break
        bar_info[empty_idx, 0] = avail_notes[i]
        
    return bar_info
    
# real application
for bar_idx in range(midi_info.shape[0]):
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        if (-1 in midi_info[bar_idx, channel_idx][:, 0]):
            midi_info[bar_idx, channel_idx] = bar_add_pitch(midi_info[bar_idx, channel_idx], key_predict, chord_predict[bar_idx])

# example: fill -1 pitch
# create five notes with -1 pitch
example_key_predict = 14
example_chord_predict = 57
example_bar_info = np.array([
    [41, 88, 0, 0.184, 0, 0.738, 0],
    [-1, 88, 0, 0.184, 0, 0.738, 0],
    [-1, 88, 0, 0.184, 0, 0.738, 0],
    [-1, 88, 0, 0.184, 0, 0.738, 0],
    [-1, 88, 0, 0.184, 0, 0.738, 0],
    [53, 88, 0.188, 0.059, 0.75, 0.237, 0],
    [41, 88, 0.25, 0.184, 1, 0.738, 0],
    [-1, 88, 0.25, 0.75, 1, 0.738, 0],
    [53, 88, 0.438, 0.059, 1.75, 0.237, 0],
    [38, 88, 0.5, 0.184, 2, 0.738, 0],
    [50, 88, 0.688, 0.059, 2.75, 0.237, 0],
    [38, 88, 0.75, 0.184, 3, 0.738, 0],
    [50, 88, 0.938, 0.059, 3.75, 0.237, 0],
])
with np.printoptions(precision=3, suppress=True):
    print("key:\t%s" % name_key(example_key_predict))
    print("bar:\t%s" % name_chord(example_chord_predict))
    print("before:\t%s" % example_bar_info[:, 0])
    print("after:\t%s" % bar_add_pitch(example_bar_info, example_key_predict, example_chord_predict)[:, 0])

key:	D minor
bar:	A7
before:	[41. -1. -1. -1. -1. 53. 41. -1. 53. 38. 50. 38. 50.]
after:	[41. 45. 50. 38. 48. 53. 41. 45. 53. 38. 50. 38. 50.]


In [298]:
# get the overall statistic of dynamics in each channel ([min, max, median])
def get_dynamics_info(midi_info):
    dynamics_info = np.zeros((MAX_NUM_OF_CHANNELS, 4), dtype=np.int16)
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        notes_dynamics = []
        for bar_idx in range(midi_info.shape[0]):
            for bar_info in midi_info[bar_idx, channel_idx]:
                notes_dynamics.append(bar_info[1])
        if len(notes_dynamics) > 0:
            dynamics_info[channel_idx] = [np.min(notes_dynamics), np.max(notes_dynamics), np.median(notes_dynamics), np.std(notes_dynamics)]
    return dynamics_info

def get_dynamics_lookup_table(dynamics_info, wide_factor=1, avg_addition_boost=0, avg_multiply_boost=1):
    dynamics_lookup_table = np.zeros((MAX_NUM_OF_CHANNELS, 128), dtype=np.int16)
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        dmin = dynamics_info[channel_idx, 0]
        dmax = dynamics_info[channel_idx, 1]
        dmed = dynamics_info[channel_idx, 2]
        for i in range(dmin, dmax + 1):
            note_velocity = 0
            if i < dmed:
                note_velocity = (int)(dmed - (dmed - dmin) * np.power((dmed - i) / (dmed - dmin), wide_factor) * avg_multiply_boost)
            else:
                if dmax == dmed:
                    note_velocity = i
                else:
                    note_velocity = (int)(dmed - (dmed - dmax) * np.power((dmed - i) / (dmed - dmax), wide_factor) * avg_multiply_boost)
            dynamics_lookup_table[channel_idx, i] = np.max((0, np.min((127, note_velocity + avg_addition_boost))))
    return dynamics_lookup_table

dynamics_info = get_dynamics_info(midi_info)
# print(dynamics_info)
dynamics_lookup_table = get_dynamics_lookup_table(dynamics_info, 2, 10)
# print(dynamics_lookup_table[5,:])
 

def get_dynamics_angry_factors(dynamics_info, wide_para_1=0.3, wide_para_2=0.1, addition_para_1=0, multiply_para_1=0.2, multiply_para_2=0):
    dynaics_angry_factors = np.zeros((MAX_NUM_OF_CHANNELS, 3))
    dynaics_angry_factors[:, 0] = 1 - 1 / (dynamics_info[:, 3] * wide_para_1 + wide_para_2 + 1)
    dynaics_angry_factors[:, 1] = np.repeat(addition_para_1, MAX_NUM_OF_CHANNELS)
    dynaics_angry_factors[:, 2] = 1 + multiply_para_1 / (dynamics_info[:, 3] + multiply_para_2 + 1)
    return dynaics_angry_factors

def get_dynamics_calm_factors(dynamics_info, wide_para_1=0.8, addition_para_1=-10, multiply_para_1=0.2, multiply_para_2=0.5):
    dynaics_calm_factors = np.zeros((MAX_NUM_OF_CHANNELS, 3))
    dynaics_calm_factors[:, 0] = 1 + dynamics_info[:, 3] * wide_para_1
    dynaics_calm_factors[:, 1] = np.repeat(addition_para_1, MAX_NUM_OF_CHANNELS)
    dynaics_calm_factors[:, 2] = 1 / (dynamics_info[:, 3] * multiply_para_1 + 1) * (1 - multiply_para_2) + multiply_para_2
    return dynaics_calm_factors
    
def get_dynamics_happy_factors(dynamics_info, wide_para_1=5, wide_para_2=0.8, wide_para_3=0.5, addition_para_1=0, multiply_para_1=0.2, multiply_para_2=0):
    dynaics_happy_factors = np.zeros((MAX_NUM_OF_CHANNELS, 3))
    for i in range(MAX_NUM_OF_CHANNELS):
        if dynamics_info[i, 3] < wide_para_1:
            dynamics_happy_factors[i, 0] = 1 - wide_para_2 * np.power((wide_para_1 - dynamics_info[i, 3]) / wide_para_1, wide_para_2)
        else:
            dynamics_happy_factors[i, 0] = (1 - 1 / ((dynamics_info[i, 3] - wide_para_1) * wide_para_3 + 1)) * wide_para_2 + 1
    dynaics_happy_factors[:, 0] = 1 - 1 / (dynamics_info[:, 3] * wide_para_1 + wide_para_2 + 1)
    dynaics_happy_factors[:, 1] = np.repeat(addition_para_1, MAX_NUM_OF_CHANNELS)
    dynaics_happy_factors[:, 2] = 1 + multiply_para_1 / (dynamics_info[:, 3] + multiply_para_2 + 1)
    return dynaics_happy_factors

def dynamics_angry(bar_info, dynamic_info, angry_dynamic_lookup_table, bar_notes_priority):
    if len(bar_info) == 0:
        return np.array([])    
    if np.min(bar_info[:,1]) != np.max(bar_info[:,1]) or len(bar_info) == 1:
        return angry_dynamic_lookup_table[[(int)(velocity) for velocity in bar_info[:, 1]]]
    else:
        dynamics_new = np.array([
            np.round(dynamic_info[0] + (dynamic_info[1] - dynamic_info[0]) / 5),
            np.round(dynamic_info[0] + (dynamic_info[1] - dynamic_info[0]) / 5 * 2),
            np.round(dynamic_info[0] + (dynamic_info[1] - dynamic_info[0]) / 5 * 3),
            np.round(dynamic_info[0] + (dynamic_info[1] - dynamic_info[0]) / 5 * 4)
        ], dtype=np.int8)
        return angry_dynamic_lookup_table[dynamics_new[bar_notes_priority]]

def dynamics_calm(bar_info, calm_dynamic_lookup_table):
    return calm_dynamic_lookup_table[[(int)(velocity) for velocity in bar_info[:, 1]]]
    
# example
example_key_predict = 14
example_chord_predict = 57
example_bar_info_1 = np.array([
    [41, 90, 0, 0.184, 0, 0.738, 0],
    [53, 70, 0.188, 0.059, 0.75, 0.237, 0],
    [41, 80, 0.25, 0.184, 1, 0.738, 0],
    [53, 60, 0.438, 0.059, 1.75, 0.237, 0],
    [38, 90, 0.5, 0.184, 2, 0.738, 0],
    [50, 70, 0.688, 0.059, 2.75, 0.237, 0],
    [38, 80, 0.75, 0.184, 3, 0.738, 0],
    [50, 60, 0.938, 0.059, 3.75, 0.237, 0],
])
example_bar_info_2 = np.array([
    [41, 88, 0, 0.184, 0, 0.738, 0],
    [53, 88, 0.188, 0.059, 0.75, 0.237, 0],
    [41, 88, 0.25, 0.184, 1, 0.738, 0],
    [53, 88, 0.438, 0.059, 1.75, 0.237, 0],
    [38, 88, 0.5, 0.184, 2, 0.738, 0],
    [50, 88, 0.688, 0.059, 2.75, 0.237, 0],
    [38, 88, 0.75, 0.184, 3, 0.738, 0],
    [50, 88, 0.938, 0.059, 3.75, 0.237, 0],
])
example_dynamic_info = np.array([60, 100, 88, 15])
example_dynamics_info = np.tile(example_dynamic_info, (16, 1))
example_bar_notes_priority = get_bar_notes_priority(example_bar_info, example_key_predict, example_chord_predict)
example_angry_dynamic_factors = get_dynamics_angry_factors(example_dynamics_info)[0]
example_calm_dynamic_factors = get_dynamics_calm_factors(example_dynamics_info)[0]
example_angry_dynamic_lookup_table = get_dynamics_lookup_table(example_dynamics_info, wide_factor=example_angry_dynamic_factors[0], avg_addition_boost=example_angry_dynamic_factors[1], avg_multiply_boost=example_angry_dynamic_factors[2])[0]
example_calm_dynamic_lookup_table = get_dynamics_lookup_table(example_dynamics_info, wide_factor=example_calm_dynamic_factors[0], avg_addition_boost=example_calm_dynamic_factors[1], avg_multiply_boost=example_calm_dynamic_factors[2])[0]
example_dynamics_angry_1 = dynamics_angry(example_bar_info_1, example_dynamic_info, example_angry_dynamic_lookup_table, example_bar_notes_priority)
example_dynamics_angry_2 = dynamics_angry(example_bar_info_2, example_dynamic_info, example_angry_dynamic_lookup_table, example_bar_notes_priority)
example_dynamics_calm_1 = dynamics_calm(example_bar_info_1, example_calm_dynamic_lookup_table)
example_dynamics_calm_2 = dynamics_calm(example_bar_info_2, example_calm_dynamic_lookup_table)
with np.printoptions(precision=3, suppress=True):
    print("angry dynamic lookup table:\n%s" % example_angry_dynamic_lookup_table)
    print("calm dynamic lookup table:\n%s" % example_calm_dynamic_lookup_table)
    print("ex.1 before:\t%s" % example_bar_info_1[:, 1])
    print("angry1 after:\t%s" % example_dynamics_angry_1)
    print("calm1 after:\t%s" % example_dynamics_calm_1)
    print("ex.2 before:\t%s" % example_bar_info_2[:, 1])
    print("angry2 after:\t%s" % example_dynamics_angry_2)
    print("calm2 after:\t%s" % example_dynamics_calm_2)
    
chord_idx = 24
chords_notes[chord_idx]

angry dynamic lookup table:
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0  59  60  61  62  63  63  64  65  66  67  68  69
  70  71  71  72  73  74  75  76  77  78  80  81  82  83  84  86  88  89
  90  91  92  93  94  95  96  97  98  99 100   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
calm dynamic lookup table:
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0 60 67 71 73 75 76 77 77 77 77 77 77
 77 77 77 77 77 77 77 77 77 77 77 77 77 77 77 78 78 78 78 78 78 78 78 78
 78 78 78 80 85  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
ex.1 before:	[90. 70. 80. 60. 90.

array([2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1], dtype=int8)

In [312]:
# chords element, '[C, Db, D, ..., B, Cm, Dbm, ..., Bbm, Bm]'
chords_elements = np.empty((0, 12), dtype=np.int8)

chords_elements_entries = np.array([
    # C1
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    # C5
    [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    # C
    [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    # Cm
    [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
    # C7
    [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
    # CMaj7
    [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    # Cmin7
    [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
    # Cdim
    [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
], dtype=np.int8)

for chords_elements_entry in chords_elements_entries:
    for i in range(12):
        chords_elements = np.vstack((chords_elements, np.roll(chords_elements_entry, i)))

def is_note_in_chord(note, chord):
    return chords_elements[chord][note % 12] == 1
        
example_chord = 44
example_note = 51
print("Chord:\t%s" % name_chord(example_chord))
print("Note:\t%s" % note_int2char(example_note))
print ("Is in chord?\t%s" % is_note_in_chord(example_note, example_chord))

Chord:	Abm
Note:	Eb
Is in chord?	True
